In [ ]:
# pip install finance-datareader
# pip install beautifulsoup4
# pip install plotly

import FinanceDataReader as fdr
import openai
import json

INIT_PROMPT = """
            너는 엔비디아 주가 정보를 분석하는 봇이야. 
            질문에 따라 답변을 해줘
            날짜는 년도만 말할 수도있고, 년 + 월, 또는 년 + 월 + 일로 질문할 수 있어 
            '2024년' 이라고 연도만 말한 경우에는 시작일은 2024년 1월 1일, 종료일은 2024년 12월31일
            '2024년 1월' 이라고 말한 경우에는 시작일은 2024년 1월 1일, 종료일은 2024년 1월 31일
            '2024년 1월 1일' 이라고 말한 경우에는 시작일은 2024년 1월 1일, 종료일은 2024년 1월 1일이야
            
            1. 엔비디아 외 회사의 주가 정보를 물어본 경우
              -> 저는 엔비디아의 주가 정보만 대답할 수 있습니다.
           
            2. 질문에 날짜가 포함되어있는 경우 JSON형식으로 대답, 날짜는 연도만 포함되어도 됨

            {
                start_dt: 사용자가 말한 날짜의 시작일 (YYYY-MM-DD 형식)
                end_dt: 사용자가 말한 날짜의 종료일 (YYYY-MM-DD 형식)
            } 

            3. 날짜를 질문하지 않은 경우 (연도 또는 월 또는 일이 포함되지 않은 경우임, 연도만 포함되어도 됨)
              -> 날짜를 입력해 주세요

            4. 그 외 엉뚱한 질문인 경우
              -> 무슨 말인지 모르겠어요. 다시 질문해 주세요.

"""

def is_valid_json(data):
    try:
        json.loads(data)
        return True
    except (TypeError, ValueError):
        return False

def chat_create(question, prompt):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": prompt
            },
            {
                "role": "user", 
                "content": question
            }],
        )
    return response.choices[0].message.content

def ask(question, prompt, cache):
    if(prompt != ""):
       cache = chat_create(question, prompt)
       return ask(question, "", cache)
    else:
        if not is_valid_json(cache):
            return cache

        ref = json.loads(cache)
        print(ref)
        start_dt = ref["start_dt"]
        end_dt = ref["end_dt"]
        df = fdr.DataReader('NVDA', start_dt, end_dt)
        df_dict = df.to_dict(orient="records")
        df_str = json.dumps(df_dict, ensure_ascii=False)

        question = (
            "질문과 데이터를 바탕으로 분석해서 결과를 알려줘"
            + "질문 : " + question
            + ", 데이터 : " + df_str
        )
        return chat_create(question, "")

def chat():
    print("날짜를 입력하시면 엔비디아 주가를 분석해드려요")
    while True:
        user_input = input("User: ")
        print(f"USER: {user_input}")

        if user_input.lower() == 'exit':
            print("Exiting...")
            break
        response = ask(user_input, INIT_PROMPT, "")
        print(f"GPT: {response}")

if __name__ == "__main__":
    chat()



날짜를 입력하시면 엔비디아 주가를 분석해드려요
USER: 2024sus
GPT: 무슨 말인지 모르겠어요. 다시 질문해 주세요.
USER: 2024년도
GPT: 날짜를 입력해 주세요.


In [28]:
import FinanceDataReader as fdr
import openai
import json

df = fdr.DataReader('NVDA', '2024-01-04', '2024-01-04')
print(df)

                 Open       High        Low      Close     Volume  Adj Close
2024-01-03  47.485001  48.183998  47.320000  47.569000  320896000  47.555515
2024-01-04  47.766998  48.500000  47.507999  47.998001  306535000  47.984398
